In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import sklearn
import tensorflow as tf
import scikeras
from tensorflow.keras import backend as k
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential


In [ ]:
#--------------------------------------------- carregando a base de dados ---------------------------------------------#

#previsores
X = pd.read_csv('entradas_breast.csv')

#classe
y = pd.read_csv('saidas_breast.csv')

In [ ]:
#--------------------------------------------- criando estrutura da rede neural ---------------------------------------------#

def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
    k.clear_session()
    rede_neural = Sequential(
    [
        tf.keras.layers.InputLayer(shape = (30,)), #camada de entrada com 30 neurônios
    
        #camada oculta com 16 neurônios 30 unidades de entrada + 1 unidade de saída / 2 = 16(arrendondado)
        tf.keras.layers.Dense(units = neurons, activation = activation, kernel_initializer = kernel_initializer ),
        tf.keras.layers.Dropout(rate = 0.2), #zera 20% dos neuronios
        tf.keras.layers.Dense(units = neurons, activation = activation, kernel_initializer = kernel_initializer ),
        tf.keras.layers.Dropout(rate = 0.2), #zera 20% dos neuronios
        tf.keras.layers.Dense(units = 1, activation = 'sigmoid'), #camada de saída com 1 neurônio
    ]
        )
    rede_neural.compile(optimizer = optimizer, loss = loss, metrics = ['binary_accuracy']) # adam = algoritmo de otimização
                                                                                                             # binary_crossentropy = função de perda para classificação binária
                                                                                                             # loss = função de perda
    return rede_neural

In [ ]:
rede_neural = KerasClassifier(model = criar_rede)

In [ ]:
#---------------------muito demorado------------------------#
parametros = {
    'batch_size': [10,30],
    'epochs': [50,100],
    'model__optimizer': ['adam', 'sgd'],
    'model__loss': ['binary_crossentropy', 'hinge'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu', 'tanh'],
    'model__neurons': [16,8]
}

In [ ]:
#------------menos variação para executar mais rapido-------------------#
parametros = {
    'batch_size': [10,30],
    'epochs': [50],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'],
    'model__kernel_initializer': ['random_uniform', 'normal'],
    'model__activation': ['relu'],
    'model__neurons': [16]
}

In [ ]:
print(parametros)

In [ ]:
grid_search = GridSearchCV(estimator = rede_neural, param_grid = parametros,
                           scoring = 'accuracy', cv =5)

In [ ]:
grid_search = grid_search.fit(X,y)

In [ ]:
print(grid_search)

In [ ]:
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

In [ ]:
melhor_precisao = grid_search.best_score_
print(melhor_precisao)